In [1]:
import pandas as pd
import numpy as np
import os

pd.set_option("mode.copy_on_write", True)

os.chdir("/Users/danielajimenez/Desktop/MIDS S1/Practical Data Science/PROJECT/")

In [2]:
# start dask cluster
import os
import dask

print(f"I have {os.cpu_count()} logical cores.")

from dask.distributed import Client

client = Client()
client

I have 8 logical cores.


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 24.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52268,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 24.00 GiB
Comm: tcp://127.0.0.1:52279,Total threads: 2
Dashboard: http://127.0.0.1:52282/status,Memory: 6.00 GiB
Nanny: tcp://127.0.0.1:52271,


In [3]:
import dask.dataframe as dd

df_dask = dd.read_csv(
    "arcos_all_washpost.tsv",
    sep="\t",
    usecols=[
        "MME_Conversion_Factor",
        "CALC_BASE_WT_IN_GM",
        "REPORTER_NAME",
        "BUYER_COUNTY",
        "BUYER_STATE",
        "TRANSACTION_DATE",
        "MME",
    ],
    dtype={
        "REPORTER_NAME": "string",
        "BUYER_COUNTY": "string",
        "BUYER_STATE": "string",
        "CALC_BASE_WT_IN_GM": "float64",
        "MME_Conversion_Factor": "float64",
        "TRANSACTION_DATE": "string",
        "MME": "float64",
    },
)

In [4]:
# create year var and group by county, state, and year to sum MME, MME conversion factor and CALC_BASE_WT_IN_GM
df_dask["YEAR"] = df_dask["TRANSACTION_DATE"].str.slice(0, 4)
df_final = (
    df_dask.groupby(["BUYER_COUNTY", "BUYER_STATE", "YEAR"])
    .agg({"MME": "sum", "MME_Conversion_Factor": "sum", "CALC_BASE_WT_IN_GM": "sum"})
    .compute()
)

In [5]:
# reset index.
df_final = df_final.reset_index()
df_final  # this would the dataframe to use in our analysis

,BUYER_COUNTY,BUYER_STATE,YEAR,MME,MME_Conversion_Factor,CALC_BASE_WT_IN_GM
0,JOHNSON,IN,2011,7.875693e+07,21730.0,63536.669741
1,JOHNSON,IN,2012,8.216889e+07,21584.0,66122.818179
2,JOHNSON,IN,2013,7.689530e+07,20188.0,61911.880688
3,JOHNSON,IN,2014,6.782561e+07,16066.0,54045.860376
4,SPENCER,IN,2012,6.885722e+06,2802.0,5946.688152
...,...,...,...,...,...,...
43147,EUREKA,NV,2007,4.540500e+02,1.0,0.454050
43148,WABAUNSEE,KS,2013,3.027000e+02,1.0,0.302700
43149,LINCOLN,ID,2006,1.195665e+03,5.0,1.195665
43150,ADAMS,ID,2008,3.632400e+03,6.0,3.632400
